In [1]:
%matplotlib qt
#%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby
import copy
import itertools
import distance
import nltk
#nltk.download('stopwords')
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from PIL import Image
import matplotlib.patches as patches

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

from itertools import *
from operator import *

# import other jupyter notebooks
from openpyxl import load_workbook

In [2]:
# exceptional removal of particular extra sentences not typed by the user 
dict_phraseStim = {
    '2019-01-30-11-22-25_1' : [4, 6, 7], # ys, session 4
    '2019-01-31-09-37-5_2ndPart_2' : range(1,5), # bh1, session 4 , all sentences except the first one deleted
    #'2019-02-05-14-10-39_2ndPart_2' : [1, 2, 3, 4, 5, 6, 9, 10],
    #'2019-01-14-14-58-30' : [0], # ys, session_trial ()
    '2019-01-16-16-36-17_1stPart_2' : [-1], # af_session1
    '2019-01-16-17-00-12_2ndPart_2': [1], # af_session1
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-17-16-03-27_2ndPart_2' : [0, 1, 2], # Af session2
    '2019-02-06-11-25-41_1' : [7],               # aq_session1    
    '2019-02-08-11-33-53_1stPart_1' : [1],  # aq session3_1_part1
    '2019-02-08-12-11-34_2ndPart_1' : [0, 1, 2, 3],  # aq session3_1_part2
    '2019-01-31-09-22-49_1stPart_2' : [4],  # bh1_session4_2_part1
    '2019-02-14-13-28-20_1stPart_2' : [2], # ch_session3_2_part1
    '2019-02-14-13-57-41_2ndPart_2' : [0, 2, 3], # ch_session3_2_part2
    '2019-01-14-15-07-21_1' : [4], # ys_session1
    '2019-01-16-15-18-50_1stPart_1' : [3, 4], # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [2], # ys_session2
    '2019-01-30-11-22-25_1' : [3, 5, 7],          # ys_session4
    '2019-01-16-15-18-0_1' : [4],            # no_session1
    '2019-02-21-16-09-44_1stPart_1' : [1], # bh2_session1
    '2019-02-21-16-22-22_2ndPart_1' : [2, 3, 4],# bh2_session1
    '2019-02-28-17-03-53_1stPart_2' : [2],       # bh2_session3
    '2019-02-28-17-24-2_2ndPart_2' : [0, 2],     # bh2_session3
    '2019-02-21-15-01-4_1stPart_1' : [0],        # le_session3
    '2019-02-21-15-25-56_2ndPart_1' : [1],        # le_session3
    '2019-02-18-10-28-35_2' : [0],               # ls2_session4
    '2019-02-05-14-00-27_1stPart_2' : [3],        # mh_session1
    '2019-02-05-14-10-39_2ndPart_2' : [0, 1, 3],   # mh_session1
    '2019-02-08-10-51-3_1stPart_1' : [4],        # mn_session1
    '2019-02-08-11-05-7_2ndPart_1' : [0, 2, 3, 4], # mn_session1
    '2019-02-19-10-34-7_1stPart_1' : [3],          # mn_session3
    '2019-02-19-10-56-43_2ndPart_1' : [1, 2, 3, 4], # mn_session3
    '2019-01-29-13-25-4_1' : [3],        # ph_session2
    '2019-03-07-16-44-5_2' : [1],                   # rh_session1
    '2019-03-14-13-56-56_2' : [2],                  # rh_session3
    '2019-02-19-17-10-45_1' : [3]                  # ph_session5
}

# exceptional removal of sentences/words typed by the user, but then deleted everything to have a blank scratchpad

dict_phraseUser = {
    "2019-02-06-15-44-15_1" : [2, 3, 6], 
    "2019-02-06-16-19-9_2" : [1, 3, 6, 7],
    "2019-02-12-11-21-21_2" : [0],
    "2019-02-14-14-28-49_1" : [0, 2, 3], # ac_session3_1
    "2019-02-14-14-45-49_2" : [0, 5, 6], # ac_session3_2
    '2019-01-29-14-19-26_1' : [0, 3, 4], # bh1_session2_1
    '2019-01-29-14-40-36_2' : [0, 1, 2], # bh1_session2_2
    '2019-01-30-14-29-29_2' : [4],       # bh1_session3_2
    '2019-01-31-09-12-2_1' : [3],         # bh1_session4_1
    '2019-01-31-09-22-49_1stPart_2' : [4], # bh1_session4_2_part1
    '2019-03-05-09-15-11_1' : [1],         # bh2_session5_1
    '2019-03-05-09-15-11_2' : [1],        # bh2_session5_2
    '2019-02-21-15-55-56_2' : [2],       # ch_session5_2
    '2019-01-30-15-19-36_2' : [1],       # jm_session2_1
    '2019-01-30-15-04-30_1' : [0],         # jm_session2_2
    '2019-01-16-15-18-50_1stPart_1' : [1],  # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [0], # ys_session2
    '2019-01-30-11-22-25_1' : [2, 4],       # ys_session4
    '2019-01-30-11-57-3_2' : [0] ,          # ys_session4
    '2019-01-31-13-13-2_1' : [4],           # ys_session5
    '2019-01-30-10-20-32_1' : [0, 1, 2, 3, 4, 5], # no_session4
    '2019-01-30-10-46-38_2' : [0],          # 
    '2019-02-28-17-03-53_1stPart_2' : [2],   # bh2_session3
    '2019-03-12-09-30-5_1' : [0],            # kj_session3
    '2019-02-13-15-20-38_1' : [0, 1, 2, 3, 6], # ls1_session3
    '2019-02-18-10-25-52_1' : [1],              # ls2_session4
    '2019-02-18-10-46-26_2' : [0],            # ls2_session4
    '2019-01-29-13-25-4_1' : [0, 1, 7],        # ph_session2
    '2019-01-29-13-43-50_2' : [0],              # ph_session2
    '2019-03-07-16-17-30_1' : [0],              # rh_session1
    '2019-03-07-16-44-5_2' : [0, 1],         # rh_session1
    '2019-03-14-13-56-56_2' : [0, 1, 3]         # rh_session3
}

# key selection can have extra selections of NextPhrase at the end
dict_keySelectionOfNextPhrase = {
    "2019-02-11-11-18-30_1" : [12, 13], # ac_session1
    "2019-01-16-17-00-12_2ndPart_2" : [12], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [12], # af_session2
    "2019-02-06-16-19-9_2" : [12], # af_session3
    "2019-02-12-11-07-43_1" : [12], # af_session4
    "2019-02-27-15-08-32_1" : [12], # af_session5
    "2019-01-28-14-30-44_1" : [12], # bh1_session1
    "2019-02-21-16-22-22_2ndPart_1" : [12], # bh2_session1
    "2019-02-18-14-02-56_2" : [12], # le_session1
    "2019-02-19-10-03-14_1" : [12], # le_session2
    "2019-02-08-11-05-7_2ndPart_1" : [12], # mn_session1
    "2019-02-08-11-12-51_2" : [12, 13], # mn_session1
    "2019-02-15-11-38-22_1" : [12, 13], # mn_session2
    "2019-02-15-11-54-25_2" : [12], # mn_session2
    "2019-01-16-15-18-0_1" : [12], # no_session1
    "2019-01-28-13-31-51_1" : [12], # ph_session1
    "2019-01-28-13-49-14_2" : [12], # ph_session1
    "2019-01-14-15-07-21_1" : [12], # ys_session1
    "2019-01-17-15-05-1_1" : [12], # ys_session3
    "2019-01-30-11-22-25_1" : [12], # ys_session4
    "2019-01-31-13-32-2_2" : [12], # ys_session5
}


# key selection when participants skips some sentences
dict_keySelectionNotCompleted = {
    "2019-01-16-16-36-17_1stPart_2" : [0, 1, 3, 5, 7], # af_session1 ---- last sentence is not finished
    "2019-01-16-17-00-12_2ndPart_2" : [0, 1, 3, 4, 5, 7, 9, 11], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [0, 1, 3, 5, 7, 9, 11], # af_session2 
    "2019-01-17-16-03-27_2ndPart_2" : [0, 1, 2, 3, 4, 5, 6, 7, 9, 11], # af_session2
    "2019-02-08-11-33-53_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # aq_session3
    "2019-02-08-12-11-34_2ndPart_1" : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11], # aq_session3
    "2019-01-31-09-22-49_1stPart_2": [0, 1, 3, 5, 7, 9, 10, 11], # bh1_session4
    "2019-01-31-09-37-5_2ndPart_2" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # bh1_session4
    "2019-02-21-16-09-44_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # bh2_session1
    "2019-02-21-16-22-22_2ndPart_1" : [0, 1, 3, 5, 6, 7, 8, 9, 10, 11], # bh2_session1
    "2019-02-28-17-03-53_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # bh2_session3
    "2019-02-28-17-24-2_2ndPart_2" : [0, 1, 2, 3, 5], # bh2_session3     ----
    "2019-02-14-13-28-20_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # cw_session3
    "2019-02-14-13-57-41_2ndPart_2" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 11], # cw_session3
    "2019-02-21-15-01-4_1stPart_1" : [0, 1, 2, 3, 5, 7, 9, 11], # le_session3
    "2019-02-21-15-25-56_2ndPart_1" : [0, 1, 3], # le_session3       ----
    "2019-02-05-14-00-27_1stPart_2" : [0, 1, 3, 5, 7, 8], # mh_session1
    "2019-02-05-14-10-39_2ndPart_2" : [0, 1, 2, 3, 4, 5, 7, 8, 9, 11], # mh_session1
    "2019-02-08-10-51-3_1stPart_1" : [0, 1, 3, 5, 7, 9, 10, 11], # mn_session1
    "2019-02-19-10-34-7_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 11], # mn_session3
    "2019-02-19-10-56-43_2ndPart_1" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # mn_session3
    "2019-01-29-13-25-4_1" : [0, 1, 3, 5, 7], # ph_session2  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-18-50_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 10], # ys_session2
    "2019-01-17-15-05-1_1" : [0, 1, 3, 5],  # ys_session3  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-02-06-11-25-41_1" : [0, 1, 3, 5, 11], # aq_session1 -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2 -- different for reading and writing, this one is for
    # writing
    '2019-01-30-11-22-25_1' : [0, 1, 3, 5, 7, 9, 11],   # ys_session4 -- sessions where there are less score questions 
    # and more sentences typed
    "2019-02-08-11-05-7_2ndPart_1" : [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11] # mn_session1 -- -- different for reading and 
    # writing, this one is for writing
}

# dictionary for phrase removal just like in the dict_phraseStim, but since not all participants require that, some that 
# do, are added to this new dictionary here
dict_keySelection_phraseStim = {
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-16-15-18-0_1' : [4],        # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-03-07-16-44-5_2' : [1],        # rh_session1
    '2019-03-14-13-56-56_2' : [2],              # rh_session3
    '2019-01-14-15-07-21_1' : [4]         # ys_session1
}


# in the beginning experiments, not everyone started with 800 initial dwell time

dict_dwellTimeOrig_not800 = {
    "2019-01-16-15-51-13_2" : 600, # no_session1
    "2019-01-16-15-18-0_1" : 600, # no_session1
    "2019-01-16-15-43-8_1" : 100, # af_session1
    "2019-01-16-16-36-17_1stPart_2" : 100, # af_session1
    "2019-01-16-17-00-12_2ndPart_2" : 100, # af_session1
    "2019-01-17-15-03-40_1" : 100, # af_session2
    "2019-01-17-15-27-20_1stPart_2" : 0, # af_session2
    "2019-01-17-16-03-27_2ndPart_2" : 100, # af_session2
    "2019-01-14-15-07-21_1" : 500, # ys_session1
    "2019-01-14-15-25-55_2" : 300, # ys_session1
    "2019-01-16-15-18-50_1stpart_1" : 200, # ys_session2
    "2019-01-16-15-42-51_2ndPart_1" : 100, # ys_session2
    "2019-01-16-15-59-55_2" : 100, # ys_session2
    "2019-01-17-15-05-1_1" : 100, # ys_session3
    "2019-01-17-15-31-12_2" : 100 # ys_session3
}


# list of all things that should be present when computing effective time
list_keysToBeCounted = ['Comma', 'BackOne', 'BackMany', 'SpaceBar']

# some sessions do not have gaze data
dict_noGazeData = {
    '2019-01-16-17-00-12_2ndPart_2' : 'no gaze data', # af_session2
    '2019-01-17-15-31-12_2' : 'no gaze data', #ys_session2
    '2019-01-30-11-57-3_2' : 'no gaze data' # ys_session4
}



In [3]:
dict_keySelection_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 3, 5], # ys_session2 
}

dict_keySelection_WritingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2   
}

# normally, reading part of trial ends when people look at the keyboardwithphrases. For some trials, this is not done,
# as the reading is done, and the trial is accidentally skipped, and written in the next trial. Here, the trial number 
# given will have the reading time ending as sleep, and not keyboard with phrases. 
dict_keyboardNotChange_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : 0, # ys_session2 
}

dict_keySelection_firstSleepNotCounted = {
    "2019-01-28-14-50-41_2" : (0, 2), # bh1_session1 -- 3rd sleep activation to be counted
    "2019-02-19-10-56-43_2ndPart_1" : 2  # mn_session3 -- 3rd sleep activation is to be counted
}

In [4]:
dict_TrialQuestionClean = {'2019-01-17-15-27-20_1stPart_2' : [11, 12], # af_2
                           '2019-01-17-16-03-27_2ndPart_2' : [1,2,3,4,5,6,7,8], # af_2
                           '2019-02-06-11-25-41_1' : [12], # aq_1
                           '2019-02-08-11-33-53_1stPart_1' : [5,6] , # aq_3
                           '2019-02-08-12-11-34_2ndPart_1' : [1,2,3,4,5,6,7,8,9,10], # aq_3
                           '2019-01-31-09-22-49_1stPart_2' : [11,12], # bh1_4
                           '2019-01-31-09-37-5_2ndPart_2' : [1,2,5,6,7,8,9,10,11,12], # bh1_4
                           '2019-02-21-16-09-44_1stPart_1': [5, 6, 8], # bh2_1
                           '2019-02-21-16-22-22_2ndPart_1': [1,2,7,8,9,10,11,12], # bh2_1
                           '2019-02-28-17-03-53_1stPart_2' : [7,8], # bh2_3
                           '2019-02-28-17-24-2_2ndPart_2': [1,2,3,4], # bh2_3
                           '2019-02-14-13-28-20_1stPart_2': [7,8], # cw_3
                           '2019-02-14-13-57-41_2ndPart_2': [1,2,3,4,7,8,9,10], # cw_3
                           '2019-02-21-15-01-4_1stPart_1': [3,4], # le_3
                           '2019-02-21-15-25-56_2ndPart_1': [1,2], # le_3
                           '2019-02-18-10-28-35_2': [1,2], # ls2_4
                           '2019-02-05-14-00-27_1stPart_2': [9], # mh_1
                           '2019-02-05-14-10-39_2ndPart_2': [1,2,3,4,5,6,9,10] , # mh_2
                           '2019-02-08-10-51-3_1stPart_1': [11,12], # mn_1
                           '2019-02-08-11-05-7_2ndPart_1': [1,2,3,4,7,8,9,10,11,12], # mn_1
                           '2019-02-19-10-34-7_1stPart_1': [9,10], # mn_3
                           '2019-02-19-10-56-43_2ndPart_1': [1,2,5,6,7,8,9,10,11,12], # mn_3
                           '2019-02-19-17-10-45_1': [9, 10], # ph_5
                           '2019-01-16-15-18-0_1': [11,12], # no_1
                           '2019-03-07-16-44-5_2': [5,6], #rh_1
                           '2019-03-14-13-56-56_2' : [7,8], #rh_3
                           '2019-01-14-15-07-21_1' : [11, 12], # ys_1
                           '2019-01-16-15-18-50_1stPart_1': [6, 9, 10, 11], # ys_2
                           '2019-01-16-15-42-51_2ndPart_1': [1, 2, 3, 4] # ys_2
                          }



dict_ChangeInTrials = {
    '2019-01-17-15-05-1_1': ['Baseline', 'PictureSentence', 'PictureDifficulty', 'TrialSentence', \
                            'TrialDifficulty', 'TrialSentence', 'TrialDifficulty', 'TrialSentence', \
                            'TrialSentence', 'TrialSentence', 'TrialSentence', 'TrialSentence',\
                            'TrialSentence'], # ys_3
    '2019-02-06-11-25-41_1': ['Baseline', 'PictureSentence', 'PictureDifficulty', 'TrialSentence', 'TrialDifficulty', \
                             'TrialSentence', 'TrialDifficulty', 'TrialSentence', 'TrialSentence', 'TrialSentence', \
                              'TrialSentence', 'TrialSentence'], # aq_1
    '2019-01-28-14-30-44_1' : ['Baseline', 'PictureSentence', 'PictureDifficulty', 'TrialSentence', \
                            'TrialDifficulty', 'TrialSentence', 'TrialDifficulty', 'TrialSentence', \
                            'TrialSentence', 'TrialSentence', 'TrialSentence', 'TrialSentence',\
                            'TrialSentence'], # bh1_1
    '2019-01-29-13-25-4_1' : ['Baseline', 'PictureSentence', 'PictureDifficulty', 'TrialSentence', \
                            'TrialDifficulty', 'TrialSentence', 'TrialDifficulty', 'TrialSentence', \
                            'TrialDifficulty', 'TrialSentence', 'TrialSentence', 'TrialSentence',\
                            'TrialSentence']
                      }

dict_ChangeInTrials_boolAddNextTrialData = {
    '2019-01-17-15-05-1_1': [False, True, False, True, False, True, False, 0, 0, 0, 0, 0, 0], # ys_3
    '2019-02-06-11-25-41_1': [False, True, False, True, False, True, False, False, False, False, False, False], # aq_1
    '2019-01-28-14-30-44_1' : [False, True, False, True, False, True, False, False, False, False, False, False, False], # bh1_1
    '2019-01-29-13-25-4_1' : [False, True, False, True, False, True, False, True, False, False, False, False, False]
                      }
                       
dict_TrialToNan = {#'2019-02-12-15-27-37_2' : [0], # ac_2_2 --> Cannot be sure
                   '2019-02-14-14-28-49_1': [7,8], # ac_3_MS_1 --> [7,8]
                   #'2019-01-16-15-43-8_1': [0], # af_1_1 --> cannot be sure
                   #'2019-01-17-15-27-20_1stPart_2' : [0], # af_2
                   #'2019-01-17-16-03-27_2ndPart_2' : [0], # af_2
                    '2019-02-19-13-31-29_2' : [5,6], # jp_5_2 --> [5,6]
                    #'2019-03-04-11-20-5_1' : [0], # le_5_MS_1 --> is looking to the picture
                    #'2019-03-04-12-00-15_2': [0], # le_5_MS_2 --> looking at the picture
                    '2019-03-07-16-17-30_1': [11, 12], # rh_1_1 --> [11, 12]
                    '2019-03-13-13-40-7_2': [3,4], # rh_2_2 --> [3,4] 
                    '2019-01-16-15-18-50_1stPart_1': [0, 6, 9, 10, 11] # ys_2 --> [0, 6, 9, 10, 11]
                    #'2019-01-31-13-13-2_1': [0] # ys_5_1 --> cannot be sure
}

                    

In [5]:
TimeDwellOrig = 800
TimeFixation = 300

In [6]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [7]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [8]:
def FixKeysSelected(KeysSelected_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    KeysSelected_New = list()
    
    # loop to combine phrases divided by commas
    KeysSelectedInd = -1 
    while KeysSelectedInd < len(KeysSelected_Old)-1:
        KeysSelectedInd = KeysSelectedInd + 1
        
        if KeysSelected_Old[KeysSelectedInd][1].count(',') > 0:
            
            keys_split = KeysSelected_Old[KeysSelectedInd][1].split("\r\n")
            del keys_split[0]
            del keys_split[-1]
            
            keys_split = [key.split(',') for key in keys_split]
            
            KeysSelected_New.extend(keys_split)
        else:
            KeysSelected_New.append(KeysSelected_Old[KeysSelectedInd])
        
    
    return KeysSelected_New

In [9]:
def ComputeDwellTime(userKeys, full_path):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dict_dwellTimeOrig_not800:
        TimeDwellOrig = dict_dwellTimeOrig_not800[session_folder_name]
    
    #print(TimeDwellOrig)
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [10]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [11]:
def findAndRemoveTrials(session_name, dictionary_saved, trials, replacingList):
    # function to check the session_name in the dictionary_saved and remove those trials from the dictionary_trial
    
    if session_name in dictionary_saved:
        index_list = dictionary_saved[session_name]
    else:
        index_list = replacingList
    
    
    
    if index_list:
        if type(trials) == list:
            for index in sorted(index_list, reverse=True):
                del trials[index]
                
        else:
            for index in sorted(index_list, reverse=True):
                del trials['start'][index]
                del trials['end'][index]
            
    return trials    

In [12]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [13]:
def timeTypingStart(userKeys):
    # From the user keys, find when the user actually starts typing, after having looked at the phrase and all the other 
    # function keys
    
    timeTypingStartInd = 0
    
    timeTypingStartIndList = list()
            
    timeUserTimeInd = 0
    
    ind = 0
    # Get start time of first trial
    
    while ind < len(userKeys):
        #print(len(userKeys[ind][1]))
        if len(userKeys[ind][1]) > 1:
            ind = ind + 1
        else:
            timeTypingStartInd = ind
            timeTypingStartIndList.append(ind)
            break
    
    #print(timeTypingStartInd)
    # Get every next phrase start timings
    while ind < len(userKeys):
        
        if userKeys[ind][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
            
            #timeTypingStartIndList.append(ind+1)
            for ind2 in range(ind+1, len(userKeys)):
                if len(userKeys[ind2][1]) > 1:
                    ind = ind + 1
                    continue
                elif userKeys[ind2][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
                    ind = ind + 1
                    continue
                else:
                    ind = ind2
                    timeTypingStartIndList.append(ind)
                    break
                    
        else:
            ind = ind + 1
            
    #print(timeTypingStartIndList)
    
    return timeTypingStartIndList

In [14]:
def gazeConvert2ColumnsTo1(GazeLog, columnIndwValidity_list, indValidity):
    # function to convert pupilsizes from 2 columns for every pupil due to comma use instead of decimal, 
    # to proper pupil sizes
    
    #columnInd_list = [joinColumn1_1, joinColumn1_2, joinColumn2_1, joinColumn2_2]
    
    # number of columns in the final dictionary
    nColumns = int(len(columnIndwValidity_list)/2)
    
    # dictionary of columns that are to be joined later
    columns_beforeDecimal = dict()
    columns_afterDecimal = dict()
    
    # dictionary of joined columns
    columnsFinal = dict()
    
    # dictionary to find and equalize missing values in every column
    missingVal_column = dict()
    missingVal = list()
    
    # find correct index of validity column to be used, to find the actual columns relative to that
    columnsValidity_inUse = list()
    
    for ind, row in enumerate(GazeLog):
        #print(ind)
        #print(sorted(list(np.where(np.array(row) == 'Valid')[0])+list(np.where(np.array(row)=='Invalid')[0]))[indValidity])

        columnsValidity = (sorted(list(np.where(np.array(row) == 'Valid')[0])+list(np.where(np.array(row)=='Invalid')[0]))[indValidity])
        columnsValidity_inUse.append(int(columnsValidity))
    
    columnsValidity_inUse = np.array(columnsValidity_inUse)
    
    columnInd_list = [[columnsValidity_inUse+i] for i in columnIndwValidity_list]
    
    
    for ind in range(0, nColumns):
        
        dict_name = 'column' + str(ind+1)
        columnsFinal[dict_name] = list()
        columns_afterDecimal[dict_name] = list()
                
        #for indItem, item4 in enumerate(GazeLog):
        #    if 'Invalid' not in item4:
        #        if columnInd_list[2*ind+1][0][indItem] < len(item4):
        #            columns_afterDecimal[dict_name].append(item4[columnInd_list[2*ind+1][0][indItem]])
        #        else:
        #            columns_afterDecimal[dict_name].append('0')
        #    else:
        #        columns_afterDecimal[dict_name].append('nan')
        
                
        columns_beforeDecimal[dict_name] = [item4[columnInd_list[2*ind][0][indItem]] if 'Invalid' not in item4 else 'nan' for indItem, item4 in enumerate(GazeLog)]
        columns_afterDecimal[dict_name] = [item4[columnInd_list[2*ind+1][0][indItem]] if 'Invalid' not in item4 and columnInd_list[2*ind+1][0][indItem] < len(item4) else 'nan' for indItem, item4 in enumerate(GazeLog)]

        
        for i in range(0, len(columns_beforeDecimal[dict_name])):
            if 'Valid' not in columns_beforeDecimal[dict_name][i] and 'Valid' not in columns_afterDecimal[dict_name][i]:
                if 'nan' not in columns_beforeDecimal[dict_name][i] and 'nan' not in columns_afterDecimal[dict_name][i]:
                    if float(columns_afterDecimal[dict_name][i]) > 0: 
                        columnsFinal[dict_name].append(float(columns_beforeDecimal[dict_name][i]+'.'+columns_afterDecimal[dict_name][i]))
                    else:
                        columnsFinal[dict_name].append(np.nan)
                else:
                    columnsFinal[dict_name].append(np.nan)
            else:
                # Rarely, the pupil size is a whole number
                columnsFinal[dict_name].append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
                # right or left eye has whole number pupil size
    
        missingVal_column[dict_name] = np.argwhere(np.isnan(columnsFinal[dict_name]))
        missingVal_column[dict_name] = list(itertools.chain.from_iterable(missingVal_column[dict_name])) # flatten the list
        
        missingVal.extend(missingVal_column[dict_name])
        
        
    
    missingVal = sorted(set(missingVal))
    
    # if one of the columns are nan, the other one is converted too
    for column in range(0, nColumns):
        dict_name = 'column' + str(column+1)
        for ind in missingVal:
            if ind < len(columnsFinal[dict_name]):
                columnsFinal[dict_name][ind] = np.nan
                
    
    
    
    #print(len(columnsFinal['column1']), len(columnsFinal['column2']))
    
    
    

    return columnsFinal

In [15]:
def EyeMovementPlot(timeBetweenTrials, GazeLog):
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # Obtain gaze location data
    # Location of eye gaze on screen in the ADCS by Tobii (in arbitrary units)
    #print('gaze point on screen')
    gazePointADCS_indWrtValidityL = [-4, -3, -2, -1]
    gazePointADCS_validityL = 3
    gazePointADCS_Left_au = gazeConvert2ColumnsTo1(GazeLog, gazePointADCS_indWrtValidityL, gazePointADCS_validityL)
    
    
    gazePointADCS_indWrtValidityR = [-4, -3, -2, -1]
    gazePointADCS_validityR = 4
    gazePointADCS_Right_au = gazeConvert2ColumnsTo1(GazeLog, gazePointADCS_indWrtValidityR, gazePointADCS_validityR)
    
    
    # gazePointADCS is in arbitrary units and needs to be converted to cm  
    screenLength = 59
    screenWidth = 34.5
    
    gazePointADCS_Left_au['column1'] = [i*59 for i in gazePointADCS_Left_au['column1']]
    gazePointADCS_Left_au['column2'] = [i*(34.5) for i in gazePointADCS_Left_au['column2']]
    
    gazePointADCS_Right_au['column1'] = [i*59 for i in gazePointADCS_Right_au['column1']]
    gazePointADCS_Right_au['column2'] = [i*(34.5) for i in gazePointADCS_Right_au['column2']]
    
    gazePointADCS = dict()
    gazePointADCS['x'] = [(v+gazePointADCS_Right_au['column1'][i])/2 for i, v in enumerate(gazePointADCS_Left_au['column1'])]
    gazePointADCS['y'] = [(v+gazePointADCS_Right_au['column2'][i])/2 for i, v in enumerate(gazePointADCS_Left_au['column2'])]
    
    gazePointBetweenTrial_x, gazePointBetweenTrial_y = list(), list() 
    
    
    for ind, inBetweenTrialStart in enumerate(timeBetweenTrials['start']):
        timeEyeDataStart, indEyeDataStart = nearestTimePoint(timeGazeLog, inBetweenTrialStart)
        timeEyeDataEnd, indEyeDataEnd = nearestTimePoint(timeGazeLog, timeBetweenTrials['end'][ind]) 
        
        print(timeEyeDataStart, timeEyeDataEnd)
        
        gazePointBetweenTrial_x = gazePointADCS['x'][indEyeDataStart:indEyeDataEnd]
        gazePointBetweenTrial_y = gazePointADCS['y'][indEyeDataStart:indEyeDataEnd]
        
        
        print('')
        # Plot the image overlay
        im = np.array(Image.open('keyboard_sleep.png'), dtype=np.uint8)
        dpi = 80
        fig, ax = plt.subplots(figsize = (20,15)) #, dpi = dpi)
        plt.axis([min(min(gazePointBetweenTrial_x), 0), max(max(gazePointBetweenTrial_x), screenLength), \
                  max(max(gazePointBetweenTrial_y), 0), min(min(gazePointBetweenTrial_y), screenWidth)])  
        
        ax.imshow(im, extent=(0, screenLength, screenWidth, 0))
        #ax.scatter(gazePointBetweenTrial_x, gazePointBetweenTrial_y)
        ax=plt.gca()                            # get the axis
        ax.set_ylim(ax.get_ylim()[::-1])        # invert the axis
        ax.xaxis.tick_top()                     # and move the X-Axis      
        ax.yaxis.tick_left()    
        
        print(len(gazePointBetweenTrial_x))
        for line in range(0,len(gazePointBetweenTrial_x)):
            if line%20 == 0:
                
                if not np.isnan(gazePointBetweenTrial_x[line]) and not np.isnan(gazePointBetweenTrial_y[line]):
                    #ax = sns.scatterplot(gazePointBetweenTrial_x[line], gazePointBetweenTrial_y[line],s = 500)
        
                    ax.text(gazePointBetweenTrial_x[line], gazePointBetweenTrial_y[line], line+1, horizontalalignment='center',\
                    size='medium', color='yellow', weight='semibold')
        
        ax.plot(gazePointBetweenTrial_x, gazePointBetweenTrial_y)
        
        break

In [16]:
# AOIs
screenLength = 59
screenWidth = 34.5
columnWidth = screenLength/11
rowHeight = screenWidth/6



Suggestion_x = [0, screenLength]
Suggestion_y = [0, rowHeight]

PhraseText_x = [2*columnWidth, 9*columnWidth]
PhraseText_y = [rowHeight, 1.5*rowHeight]

ScratchPad_x = [2*columnWidth, 9*columnWidth]
ScratchPad_y = [1.5*rowHeight, 2*rowHeight]

Keyboard_x = [0, screenLength]
Keyboard_y = [2*rowHeight, screenWidth]

NextPhrase_x = [9*columnWidth, 10*columnWidth]
NextPhrase_y = [rowHeight, 2*rowHeight]

Sleep_x = [10*columnWidth, screenLength]
Sleep_y = [rowHeight, 2*rowHeight]

aoi_plot = {'Suggestion': [(Suggestion_x[0]+Suggestion_x[1])/2, (Suggestion_y[0]+Suggestion_y[1])/2], \
            'PhraseText': [(PhraseText_x[0]+PhraseText_x[1])/2, (PhraseText_y[0]+PhraseText_y[1])/2], \
            'ScratchPad': [(ScratchPad_x[0]+ScratchPad_x[1])/2, (ScratchPad_y[0]+ScratchPad_y[1])/2], \
            'Keyboard': [(Keyboard_x[0]+Keyboard_x[1])/2, (Keyboard_y[0]+Keyboard_y[1])/2], \
            'NextPhrase': [(NextPhrase_x[0]+NextPhrase_x[1])/2, (NextPhrase_y[0]+NextPhrase_y[1])/2], \
            'Sleep': [(Sleep_x[0]+Sleep_x[1])/2, (Sleep_y[0]+Sleep_y[1])/2], \
            'OutOfScreen': [59, 34.5]
           }


In [17]:
def ComputeTimeBetweenTrials(userKeys):
    
    timeBetweenTrials = dict()
    timeBetweenTrials['start'], timeBetweenTrials['end'] = list(), list()
    
    
    
    indLast = 0
    for ind, userKey in enumerate(userKeys):
        #if len(timeBetweenTrials['start']) == 12:
        #    break
        if userKey[1] == 'NextPhrase' and userKeys[ind-1][1] != 'NextPhrase':
            if userKeys[ind-1][1] == 'Sleep' and float(userKeys[ind-1][2]) == 1:
                print('SLEEP!!!')
            #print('before next phrase')
            #print(userKeys[ind-1])
            #print(userKey)
        
        if userKey[1] == 'NextPhrase' and float(userKey[2]) == 1:
            
            time1, t1, t2 = userKey[0].partition('+')
            timeBetweenTrials['start'].append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), \
                                                                         "%Y-%m-%d-%H-%M-%S-%f"))
            #print('start: ', timeBetweenTrials['start'][-1])
            if ind < len(userKeys)-1:
                indLast = ind
                for ind2, userKey2 in enumerate(userKeys[ind:]):
                    if userKey2[1] == 'Sleep':
                        time1, t1, t2 = userKey2[0].partition('+')
                        timeBetweenTrials['end'].append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), \
                                                                         "%Y-%m-%d-%H-%M-%S-%f"))
                        #print('end: ', timeBetweenTrials['end'][-1])
                        #print('')
                        break
                        
    if len(timeBetweenTrials['start']) > len(timeBetweenTrials['end']):
        for ind3, userKey3 in enumerate(userKeys[indLast:]):
            if userKey3[1] == 'Quit':
                time1, t1, t2 = userKey3[0].partition('+')
                timeBetweenTrials['end'].append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), \
                                                                         "%Y-%m-%d-%H-%M-%S-%f"))
                #print('end: ', timeBetweenTrials['end'][-1])
                #print('')
                break
                
    
    print(len(timeBetweenTrials['start']), len(timeBetweenTrials['end']))
    return timeBetweenTrials

In [18]:
def ComputeTimeOnKeys_test(userKeys):
    
    timeBetweenTrials = dict()
    timeBetweenTrials['start'], timeBetweenTrials['end'] = list(), list()
    
    # first sleep selection:
    time1, t1, t2 = userKeys[0][0].partition('+')
    timeBetweenTrials['start'].append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), \
                                                                         "%Y-%m-%d-%H-%M-%S-%f"))
    
    time1, t1, t2 = userKeys[73][0].partition('+')
    timeBetweenTrials['end'].append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), \
                                                                         "%Y-%m-%d-%H-%M-%S-%f"))
    
    # first letter selection:
    time1, t1, t2 = userKeys[76][0].partition('+')
    timeBetweenTrials['start'].append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), \
                                                                         "%Y-%m-%d-%H-%M-%S-%f"))
    
    time1, t1, t2 = userKeys[148][0].partition('+')
    timeBetweenTrials['end'].append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), \
                                                                         "%Y-%m-%d-%H-%M-%S-%f"))
    
    
    return timeBetweenTrials

In [19]:
def EyeMovementBetweenAOI_Plot(timeBetweenTrials, GazeLog, EndPhrases, session_folder_name):
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    
    # Obtain gaze location data
    # Location of eye gaze on screen in the ADCS by Tobii (in arbitrary units)
    #print('gaze point on screen')
    gazePointADCS_indWrtValidityL = [-4, -3, -2, -1]
    gazePointADCS_validityL = 3
    gazePointADCS_Left_au = gazeConvert2ColumnsTo1(GazeLog, gazePointADCS_indWrtValidityL, gazePointADCS_validityL)
    
    
    gazePointADCS_indWrtValidityR = [-4, -3, -2, -1]
    gazePointADCS_validityR = 4
    gazePointADCS_Right_au = gazeConvert2ColumnsTo1(GazeLog, gazePointADCS_indWrtValidityR, gazePointADCS_validityR)
    
    
    # gazePointADCS is in arbitrary units and needs to be converted to cm  
    screenLength = 59
    screenWidth = 34.5
    
    gazePointADCS_Left_au['column1'] = [i*screenLength for i in gazePointADCS_Left_au['column1']]
    gazePointADCS_Left_au['column2'] = [i*screenWidth for i in gazePointADCS_Left_au['column2']]
    
    gazePointADCS_Right_au['column1'] = [i*screenLength for i in gazePointADCS_Right_au['column1']]
    gazePointADCS_Right_au['column2'] = [i*screenWidth for i in gazePointADCS_Right_au['column2']]
    
    gazePointADCS = dict()
    gazePointADCS['x'] = [(v+gazePointADCS_Right_au['column1'][i])/2 for i, v in enumerate(gazePointADCS_Left_au['column1'])]
    gazePointADCS['y'] = [(v+gazePointADCS_Right_au['column2'][i])/2 for i, v in enumerate(gazePointADCS_Left_au['column2'])]
    
    gazePointBetweenTrial_x, gazePointBetweenTrial_y = list(), list() 
    
    timeList = list()
    
    for ind, inBetweenTrialStart in enumerate(timeBetweenTrials['start']):
        
        # no session has more than 13 trials
        if ind > 12:
            break
        
        if session_folder_name in dict_TrialQuestionClean:
            
            if ind in dict_TrialQuestionClean[session_folder_name]:
                #print(session_folder_name, ' has ', str(ind))
                continue
        #else:
        #    print(session_folder_name, 'not in dict')
            
        timeEyeDataStart, indEyeDataStart = nearestTimePoint(timeGazeLog, inBetweenTrialStart)
        timeEyeDataEnd, indEyeDataEnd = nearestTimePoint(timeGazeLog, timeBetweenTrials['end'][ind]) 
        
        
        gazePointBetweenTrial_x = gazePointADCS['x'][indEyeDataStart:indEyeDataEnd]
        gazePointBetweenTrial_y = gazePointADCS['y'][indEyeDataStart:indEyeDataEnd]
        
        fixationLength = 1
        aoi_previous = None
        aoi_list, aoi_length = list(), list()
        
        flagFirst = 0
        checkFixationLength = 0
        
        flagNextPhrase = 1
        
        aoi_point, aoi_pointList = list(), list()
        
        
        for ind2 in range(0, len(gazePointBetweenTrial_x)):
            
            xPoint = gazePointBetweenTrial_x[ind2]
            yPoint = gazePointBetweenTrial_y[ind2]
            
            if np.isnan(xPoint) or np.isnan(yPoint):
                aoi_current = 'Blink'
            
            
            
            elif xPoint>ScratchPad_x[0] and xPoint<ScratchPad_x[1] and yPoint>ScratchPad_y[0] and yPoint<ScratchPad_y[1]:
                aoi_current = 'ScratchPad'    
            elif xPoint>PhraseText_x[0] and xPoint<PhraseText_x[1] and yPoint>PhraseText_y[0] and yPoint<PhraseText_y[1]:
                aoi_current = 'PhraseText'
                #print(ind2/90)
                if ind2/90 >= 5:
                    #print('New sentence appeared', ind2, ind2/90)
                    flagNextPhrase = 0
                    break
                #else:
                #    a = 1
                #    print('New sentence not yet there', timeEyeDataStart + datetime.timedelta(seconds = ind2/90))
            elif xPoint>Suggestion_x[0] and xPoint<Suggestion_x[1] and yPoint>Suggestion_y[0] and yPoint<Suggestion_y[1]:
                aoi_current = 'Suggestion'
            elif xPoint>Keyboard_x[0] and xPoint<Keyboard_x[1] and yPoint>Keyboard_y[0] and yPoint<Keyboard_y[1]:
                aoi_current = 'Keyboard'
            elif xPoint>Sleep_x[0] and xPoint<Sleep_x[1] and yPoint>Sleep_y[0] and yPoint<Sleep_y[1]:
                aoi_current = 'Sleep'
            elif xPoint>NextPhrase_x[0] and xPoint<NextPhrase_x[1] and yPoint>NextPhrase_y[0] and yPoint<NextPhrase_y[1]:
                aoi_current = 'NextPhrase'                
            else:
                aoi_current = 'OutOfScreen'
            
            #print(aoi_current)
            #if ind == 4:
            #    print(ind2, xPoint, yPoint, aoi_current)
            
            #print(aoi_previous, flagFirst)
            
            if aoi_previous != aoi_current and flagFirst: # except for the first non-match, every time the aoi does not 
                # match the previous one, the previous list, aoi, fixation time are saved for later use
                flagFirst = 1
                
                aoi_length.append(fixationLength)
                aoi_list.append(aoi_previous)
                checkFixationLength = checkFixationLength + fixationLength
                fixationLength = 1
                
                aoi_pointList.append(aoi_point)
                aoi_point = list()
                aoi_previous = aoi_current
                aoi_point.append([xPoint, yPoint])
                
                
            else:
                aoi_previous = aoi_current
                aoi_point.append([xPoint, yPoint])
                fixationLength = fixationLength + 1
                flagFirst = 1
        
        #print(ind, timeEyeDataStart, timeEyeDataStart + datetime.timedelta(seconds = ind2/90))
        #print(datetime.timedelta(seconds=(ind2/90-5)))
        timeExtra = datetime.timedelta(seconds=(ind2/90-5))     
            
        # the last aoi will not get added by the above procedure.
        aoi_length.append(fixationLength)
        aoi_list.append(aoi_previous)
        checkFixationLength = checkFixationLength + fixationLength
        aoi_pointList.append(aoi_point)
                
        
                
        
        aoi_listCombine = [[aoi, aoi_length[i]] for i, aoi in enumerate(aoi_list)]
        
        timeRest = sum([aoi_length[ind] for ind, aoi in enumerate(aoi_list) if aoi == 'Blink' or aoi =='OutOfScreen'])
        timeScratchPad = sum([aoi_length[ind] for ind, aoi in enumerate(aoi_list) if aoi == 'ScratchPad'])
        timePhraseText = sum([aoi_length[ind] for ind, aoi in enumerate(aoi_list) if aoi == 'PhraseText'])
        timeNextPhrase = sum([aoi_length[ind] for ind, aoi in enumerate(aoi_list) if aoi == 'NextPhrase'])
        timeSleep = sum([aoi_length[ind] for ind, aoi in enumerate(aoi_list) if aoi == 'Sleep'])
        timeTotal = len(gazePointBetweenTrial_x) 
        
        timeList.append([timeExtra, timeRest/timeTotal, timeScratchPad/timeTotal, timePhraseText/timeTotal, timeNextPhrase/timeTotal,\
                         timeSleep/timeTotal])
        
        #print(timeRest/len(gazePointBetweenTrial_x), timeScratchPad/len(gazePointBetweenTrial_x), \
              #timeNextPhrase/len(gazePointBetweenTrial_x), timeSleep/len(gazePointBetweenTrial_x))
        #print(EndPhrases[ind])
        
        #print('')
        
        title = EndPhrases[ind] + ': ' + str(timeRest/len(gazePointBetweenTrial_x)) + ',' +\
        str(timeScratchPad/len(gazePointBetweenTrial_x)) + ',' + str(timeNextPhrase/len(gazePointBetweenTrial_x)) + ',' + \
        str(timeSleep/len(gazePointBetweenTrial_x))
        
        
        
        """
        
        # plot 
        plt.style.use('dark_background')
        
        im = np.array(Image.open('keyboard_sleep.png'), dtype=np.uint8)
        dpi = 80
        fig, ax = plt.subplots(figsize = (20,15)) #, dpi = dpi)
        #plt.axis([min(min(gazePointBetweenTrial_x), 0), max(max(gazePointBetweenTrial_x), screenLength), \
        #          max(max(gazePointBetweenTrial_y), 0), min(min(gazePointBetweenTrial_y), screenWidth)])  
        
        plt.axis([0, 60, 0, 40])
        
        ax.imshow(im, extent=(0, screenLength, screenWidth, 0))
        #ax.scatter(gazePointBetweenTrial_x, gazePointBetweenTrial_y)
        ax=plt.gca()                            # get the axis
        ax.set_ylim(ax.get_ylim()[::-1])        # invert the axis
        ax.xaxis.tick_top()                     # and move the X-Axis      
        ax.yaxis.tick_left() 
        
        plt.title(title)
        
        
        xList, yList = list(), list()
        
        
        #print('aoi: ', aoi_listCombine)
        
        # Create a Rectangle patch
        for nNum, key in enumerate(aoi_listCombine):
            
            if key[0] == 'Blink':
                continue
            
            #print(nNum, key)
            
            xPoint = np.nanmean(np.array(([aoi[0] for aoi in aoi_pointList[nNum]])))
            yPoint = np.nanmean(np.array([aoi[1] for aoi in aoi_pointList[nNum]]))
            
            
            xList.append(xPoint)
            yList.append(yPoint)
            rect = patches.Circle([xPoint, yPoint], min(key[1]/5, 2), linewidth=1, facecolor=(0,1,1), alpha = 0.25)
            # Add the patch to the Axes
            ax.add_patch(rect)
            ax.text(xPoint, yPoint, nNum, horizontalalignment='center',size='large', color='white', weight='semibold')
            
        
        ax.plot(xList, yList, color='white', linewidth=2)
        
        #for i in aoi_listCombine:
        #    print(i)
        
        """
         
    return timeList   
    
            
    
        

In [20]:
class DataForEveryTrial:
    subjectID = ''
    blockNumber = ''
    sessionNumber = ''
    variable = ''
    dataForTrial = ''
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        startrow = writer.sheets['Sheet1'].max_row
        dataFrame.to_excel(writer, startrow = startrow, index = False, header = False)
        
        writer.save()
    

In [ ]:
metricComputed_time = 'TimeAfterTrials'
metricComputed_timeResting = 'TimeRestingAfterTrials'

resultFileName_time = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Performance\Subject_Block_Session_Trial_' + metricComputed_time +  '.xlsx'
resultFileName_timeResting = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Performance\Subject_Block_Session_Trial_' + metricComputed_timeResting +  '.xlsx'

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4


for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        #if 'ac\\3_MS\\' not in root:
        #    continue
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        #if '2019-02-06-11-25-41_1' not in root:
        #    continue
        
        
        #if 'ac\\1\\2019-02-11-11-18-30_1' not in root:
        #    continue
        
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                    scratchPad.remove[scratchPad[0]]
                    print(scratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or scratchPad is None or gazeLog is None:
            continue
        else:
                 
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            # fix phraselog due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            # trial times between NextPhrase and Sleep
            timeBetweenTrials = ComputeTimeBetweenTrials(userKeys_new)
            #timeBetweenTrials = ComputeTimeOnKeys_test(userKeys_new)
            
            timeList_scratchPadStr = [item3[0] for item3 in scratchPad_new][1:]
            # convert the list of strings to datetime formats
            timeList_scratchPad = timeConversion(timeList_scratchPadStr)
            
            endPhrase = list()
            for i, timeStart in enumerate(timeBetweenTrials['start']):
                timeNearest, indNearest = nearestTimePoint(timeList_scratchPad, timeStart)
                endPhrase.append(scratchPad_new[indNearest+1][1])
                #print(timeStart, timeNearest, scratchPad_new[indNearest+1][1])
            #if 'Part' in root:
            #    continue
            
            timeTrialSentences = list()
            timeResting = list()
            
            if sessionFolderName in dict_noGazeData:
                
                print('no gaze data present')
                timeTrialSentences = [np.nan]*5 # will check later in the file if it is correct or not 
                timeResting = [np.nan]*5
                
            else:
                # filter the data
                timeList = EyeMovementBetweenAOI_Plot(timeBetweenTrials, gazeLog, endPhrase, sessionFolderName)
                
                TrialStructure = ['Baseline', 'PictureSentence', 'PictureDifficulty', 'TrialSentence', 'TrialDifficulty', \
                 'TrialSentence', 'TrialDifficulty', 'TrialSentence', 'TrialDifficulty', 'TrialSentence', \
                  'TrialDifficulty', 'TrialSentence', 'TrialDifficulty']
              
                plt.figure()
                xP = 1
                colorList = ['blue', 'pink', 'green', 'red', 'orange']
            
                replacingList = []
                #timeList = findAndRemoveTrials(sessionFolderName, dict_TrialQuestionClean, timeList, replacingList)
                TrialStructure = findAndRemoveTrials(sessionFolderName, dict_TrialQuestionClean, TrialStructure, replacingList)
                #print(TrialStructure)
            
                
                for i, timeList2 in enumerate(timeList):
                    
                    if sessionFolderName not in dict_ChangeInTrials:
                        TrialCurrent = TrialStructure[i]
                    else:
                        TrialCurrent = dict_ChangeInTrials[sessionFolderName][i]
                    
                    if TrialCurrent == 'Baseline':
                        timeTrials = timeList2
                    elif TrialCurrent == 'PictureSentence':
                        if i+1 == len(TrialStructure):
                            timeTrials = timeList2
                        else:    
                            if TrialStructure[i+1] == 'PictureDifficulty':
                                timeTrials = list(np.array(timeList2) + np.array(timeList[i+1]))
                            else:
                                timeTrials = timeList2
                    elif TrialCurrent == 'TrialSentence':
                        if i+1 == len(TrialStructure):
                            timeTrials = timeList2
                        else:    
                            if TrialStructure[i+1] == 'TrialDifficulty':
                                timeTrials = list(np.array(timeList2) + np.array(timeList[i+1]))
                            else:
                                timeTrials = timeList2
                    else:
                        continue
                        
                        
                    if TrialCurrent == 'TrialSentence':
                        if sessionFolderName in dict_TrialToNan and i in dict_TrialToNan[sessionFolderName]:
                            print(sessionFolderName, dict_TrialToNan[sessionFolderName])
                            timeTrialSentences.append(np.nan)
                            timeResting.append(np.nan)
                        else:
                            timeTrialSentences.append(timeTrials[0].total_seconds()) # total time 
                            timeResting.append((timeTrials[0]*timeTrials[1]).total_seconds())
                            # second element of list is when people are blinking or
                        # looking out of the screen, in short, resting
                    print(i, TrialCurrent, ' : ', timeTrials[0])
                    for i2, time_ in enumerate(timeTrials[1:]):
                        #print(xP + i2, time_)
                        plt.bar(xP+i2, time_, color = colorList[i2])
                    xP = xP + 6
                plt.ylim([0,1])
                plt.close()
                
            if '1stPart' in root:
                timeTrialSentences1 = timeTrialSentences
                timeResting1 = timeResting
                continue
            if '2ndPart' in root:
                timeTrialSentences2 = timeTrialSentences
                timeResting2 = timeResting
                
                timeTrialSentences = timeTrialSentences1 + timeTrialSentences2
                timeResting = timeResting1 + timeResting2
                
                timeTrialSentences1, timeResting1 = list(), list()
                
                
            # save the total time
            dataToSave_time = DataForEveryTrial()
            dataToSave_time.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_time.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_time.sessionNumber = subjAndSessionName[-1]
            dataToSave_time.variable = metricComputed_time
            dataToSave_time.dataForTrial = timeTrialSentences
            dataToSave_time.resultPathName = resultFileName_time
            
            print(dataToSave_time.printInfo())
            
            
            
            # save the resting time
            dataToSave_timeResting = DataForEveryTrial()
            dataToSave_timeResting.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_timeResting.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_timeResting.sessionNumber = subjAndSessionName[-1]
            dataToSave_timeResting.variable = metricComputed_timeResting
            dataToSave_timeResting.dataForTrial = timeResting
            dataToSave_timeResting.resultPathName = resultFileName_timeResting
            
            print(dataToSave_timeResting.printInfo())
            
            
            #dataToSave_time.AddToFile()
            #dataToSave_timeResting.AddToFile()
            
            
            

In [ ]:
dict_TrialQuestionClean = {'2019-01-17-15-27-20_1stPart_2' : [11, 12], # af_2
                           '2019-01-17-16-03-27_2ndPart_2' : [1,2,3,4,5,6,7,8], # af_2
                           '2019-02-06-11-25-41_1' : [12], # aq_1
                           '2019-02-08-11-33-53_1stPart_1' : [5,6] , # aq_3
                           '2019-02-08-12-11-34_2ndPart_1' : [1,2,3,4,5,6,7,8,9,10], # aq_3
                           '2019-01-31-09-22-49_1stPart_2' : [11,12], # bh1_4
                           '2019-01-31-09-37-5_2ndPart_2' : [1,2,5,6,7,8,9,10,11,12], # bh1_4
                           '2019-02-21-16-09-44_1stPart_1': [5, 6, 8], # bh2_1
                           '2019-02-21-16-22-22_2ndPart_1': [1,2,7,8,9,10,11,12], # bh2_1
                           '2019-02-28-17-03-53_1stPart_2' : [7,8], # bh2_3
                           '2019-02-28-17-24-2_2ndPart_2': [1,2,3,4], # bh2_3
                           '2019-02-14-13-28-20_1stPart_2': [7,8], # cw_3
                           '2019-02-14-13-57-41_2ndPart_2': [1,2,3,4,7,8,9,10], # cw_3
                           '2019-02-21-15-01-4_1stPart_1': [3,4], # le_3
                           '2019-02-21-15-25-56_2ndPart_1': [1,2], # le_3
                           '2019-02-18-10-28-35_2': [1,2], # ls2_4
                           '2019-02-05-14-00-27_1stPart_2': [9], # mh_1
                           '2019-02-05-14-10-39_2ndPart_2': [1,2,3,4,5,6,9,10] , # mh_2
                           '2019-02-08-10-51-3_1stPart_1': [11,12], # mn_1
                           '2019-02-08-11-05-7_2ndPart_1': [1,2,3,4,7,8,9,10,11,12], # mn_1
                           '2019-02-19-10-34-7_1stPart_1': [9,10], # mn_3
                           '2019-02-19-10-56-43_2ndPart_1': [1,2,5,6,7,8,9,10,11,12], # mn_3
                           '2019-02-19-17-10-45_1': [9, 10], # ph_5
                           '2019-01-16-15-18-0_1': [11,12], # no_1
                           '2019-03-07-16-44-5_2': [5,6], #rh_1
                           '2019-03-14-13-56-56_2' : [7,8], #rh_3
                           '2019-01-14-15-07-21_1' : [11, 12], # ys_1
                           '2019-01-16-15-18-50_1stPart_1': [6, 9, 10, 11], # ys_2
                           '2019-01-16-15-42-51_2ndPart_1': [1, 2, 3, 4] # ys_2
                          }



dict_ChangeInTrials = {
    '2019-01-17-15-05-1_1': ['Baseline', 'PictureSentence', 'PictureDifficulty', 'TrialSentence', \
                            'TrialDifficulty', 'TrialSentence', 'TrialDifficulty', 'TrialSentence', \
                            'TrialSentence', 'TrialSentence', 'TrialSentence', 'TrialSentence',\
                            'TrialSentence'], # ys_3
    '2019-02-06-11-25-41_1': ['Baseline', 'PictureSentence', 'PictureDifficulty', 'TrialSentence', 'TrialDifficulty', \
                             'TrialSentence', 'TrialDifficulty', 'TrialSentence', 'TrialSentence', 'TrialSentence', \
                              'TrialSentence', 'TrialSentence'], # aq_1
    '2019-01-28-14-30-44_1' : ['Baseline', 'PictureSentence', 'PictureDifficulty', 'TrialSentence', \
                            'TrialDifficulty', 'TrialSentence', 'TrialDifficulty', 'TrialSentence', \
                            'TrialSentence', 'TrialSentence', 'TrialSentence', 'TrialSentence',\
                            'TrialSentence'], # bh1_1
    '2019-01-29-13-25-4_1' : ['Baseline', 'PictureSentence', 'PictureDifficulty', 'TrialSentence', \
                            'TrialDifficulty', 'TrialSentence', 'TrialDifficulty', 'TrialSentence', \
                            'TrialDifficulty', 'TrialSentence', 'TrialSentence', 'TrialSentence',\
                            'TrialSentence']
                      }

dict_ChangeInTrials_boolAddNextTrialData = {
    '2019-01-17-15-05-1_1': [False, True, False, True, False, True, False, 0, 0, 0, 0, 0, 0], # ys_3
    '2019-02-06-11-25-41_1': [False, True, False, True, False, True, False, False, False, False, False, False], # aq_1
    '2019-01-28-14-30-44_1' : [False, True, False, True, False, True, False, False, False, False, False, False, False], # bh1_1
    '2019-01-29-13-25-4_1' : [False, True, False, True, False, True, False, True, False, False, False, False, False]
                      }
                       
dict_TrialToNan = {#'2019-02-12-15-27-37_2' : [0], # ac_2_2 --> Cannot be sure
                   '2019-02-14-14-28-49_1': [7,8], # ac_3_MS_1 --> [7,8]
                   #'2019-01-16-15-43-8_1': [0], # af_1_1 --> cannot be sure
                   #'2019-01-17-15-27-20_1stPart_2' : [0], # af_2
                   #'2019-01-17-16-03-27_2ndPart_2' : [0], # af_2
                    '2019-02-19-13-31-29_2' : [5,6], # jp_5_2 --> [5,6]
                    #'2019-03-04-11-20-5_1' : [0], # le_5_MS_1 --> is looking to the picture
                    #'2019-03-04-12-00-15_2': [0], # le_5_MS_2 --> looking at the picture
                    '2019-03-07-16-17-30_1': [11, 12], # rh_1_1 --> [11, 12]
                    '2019-03-13-13-40-7_2': [3,4], # rh_2_2 --> [3,4] 
                    '2019-01-16-15-18-50_1stPart_1': [0, 6, 9, 10, 11] # ys_2 --> [0, 6, 9, 10, 11]
                    #'2019-01-31-13-13-2_1': [0] # ys_5_1 --> cannot be sure
}

                    

In [ ]:
def PlotPupilSizeWithEvents(PupilData_df, Event1_index, Event2_index, PlotTitle, SubjectAndSessionName, eventNumbers):
    
    subjectID = SubjectAndSessionName.split('__')[0]
    
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    
    leftPlot = ax.plot(PupilData_df['timeStamp'], PupilData_df['pupilLeft'], 'b', label = 'Left')
    rightPlot = ax.plot(PupilData_df['timeStamp'], PupilData_df['pupilRight'], 'r', label = 'Right')
    
    for start in Event1_index['start']:
        event1_start = ax.plot([PupilData_df['timeStamp'][start], PupilData_df['timeStamp'][start]], [np.min(PupilData_df['pupilLeft']), np.max(PupilData_df['pupilLeft'])], color = 'orange')
    
    for end in Event1_index['end']:
        event1_end = ax.plot([PupilData_df['timeStamp'][end], PupilData_df['timeStamp'][end]], [np.min(PupilData_df['pupilLeft']), np.max(PupilData_df['pupilLeft'])], color = 'pink')
    
    ax.set_ylabel('Absolute pupil size [in mm]')
    ax.set_xlabel('Time [in s]')
    
    plt.title('plot {}  for subject  {}'.format(PlotTitle, SubjectAndSessionName))
    
    if eventNumbers > 1:
        for start in Event2_index['start']:
            event2_start = ax.plot([PupilData_df['timeStamp'][start], PupilData_df['timeStamp'][start]], [np.min(PupilData_df['pupilLeft']), np.max(PupilData_df['pupilLeft'])], color = 'green')
    
        for end in Event2_index['end']:
            event2_end = ax.plot([PupilData_df['timeStamp'][end], PupilData_df['timeStamp'][end]], [np.min(PupilData_df['pupilLeft']), np.max(PupilData_df['pupilLeft'])], color = 'gray')
    
    #plt.legend((leftPlot, rightPlot, event1_start, event1_end, event2_start, event2_end), ('Left pupil', 'Right pupil', 'Reading start', 'Reading end', 'Writing start', 'Writing End'))
        plt.legend([leftPlot[0], rightPlot[0], event1_start[0], event1_end[0], event2_start[0], event2_end[0]], ('Left pupil', 'Right pupil', 'Reading start', 'Reading end', 'Writing start', 'Writing End'))
    else:
        plt.legend([leftPlot[0], rightPlot[0], event1_start[0], event1_end[0]], ('Left pupil', 'Right pupil', 'Trial start', 'Trial end'))
    
    
    pickle.dump(fig, open(r'C:\DTU\Results\201901_Expt' + '\\Expt_w' + plotTitle + '\\' + subjectID + '\\' + 'pupilSize_CompleteExperiment_w' + PlotTitle + '_' + SubjectAndSessionName, 'wb'))
    
    plt.close()

In [ ]:
def FindAndPlotPupilSizeForEpoch(GazeLog, TimeEpochTrial):
    # function that uses the list of start and end trial times to find the pupil sizes for those trials and plots them
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # internal time, to depict seconds
    timeInternalGazeLog = [float(item3[1]) for item3 in GazeLog]
    
    # extract pupil sizes in decimals from the strange 2 columns for every pupil
    pupilLogL, pupilLogR = Convert2ColumnSizesTo1(GazeLog)
    
    timeOfGaze_TrialList = list()
    
    
    blinkDurationAverageList = list()
    blinkTimeList = list()
    blinkFrequencyList = list()
    timeInS_List = list()
    pupilMean_Absolute = list()
    pupilMean_Relative = list()
    interBlinkDurationList = list()
    
    # for every epoch, plot the pupil size
    for trialNr in range(0, len(TimeEpochTrial['start'])):
        
        # find pupil sizes for the trial
        pupilSizeL_Trial, pupilSizeR_Trial, timeGaze_Trial, timeInternal_Trial = PupilSizeFromTrialTimes(
            [TimeEpochTrial['start'][trialNr], TimeEpochTrial['end'][trialNr]], timeGazeLog, 
                                timeInternalGazeLog, pupilLogL, pupilLogR)
        
        pupilSize_Trial = dict()
        pupilSize_Filter = dict()
        pupilSize_woBlink = dict()
        
        # find difference in consecutive elements of internal time
        timeInternalDifference = [t - s for s, t in zip(timeInternal_Trial, timeInternal_Trial[1:])]
        # divide by 1000 to make it s
        timeOfGaze_Trial = [sum(timeInternalDifference[:i])/1000000 for i in range(1,len(timeInternalDifference))]

        
        pupilSize_Trial['Left'] = pupilSizeL_Trial
        pupilSize_Trial['Right'] = pupilSizeR_Trial
        
        
        # filter the blinks
        pupilSizeL_woBlink, time_filter, blinkDuration, blinkFrequency, interBlinkDuration, timeInS_filter = filterBlinks(pupilSizeL_Trial, timeGaze_Trial, timeOfGaze_Trial)
        pupilSizeR_woBlink, time_filter, blinkDuration, blinkFrequency, interBlinkDuration, timeInS_filter = filterBlinks(pupilSizeR_Trial, timeGaze_Trial, timeOfGaze_Trial)
        
        # time of trial
        timeInS_List.append(timeInS_filter)
        
        pupilSize_woBlink['Left'] = pupilSizeL_woBlink
        pupilSize_woBlink['Right'] = pupilSizeR_woBlink
        
        # Hampel filter to remove the outliers
        winSize = 25
        pupilSizeL_filter = hampel(pupilSizeL_woBlink, winSize, 3, False)
        pupilSizeR_filter = hampel(pupilSizeR_woBlink, winSize, 3, False)

        pupilSize_Filter['Left'] = pupilSizeL_filter
        pupilSize_Filter['Right'] = pupilSizeR_filter
        
        
        RLCorrelation = np.corrcoef(pupilSizeL_filter, pupilSizeR_filter)
        if RLCorrelation[0][1] < 0.8:
            print('correlation between left and right:  ', RLCorrelation)
        
        timeOfGaze_TrialList.append(timeOfGaze_Trial[-1])
        
        timeInS_trialFilteredPlot = np.arange(0, len(pupilSize_Filter['Left'])/90, 1/90)
        timeOfGaze_trialNonFilteredPlot = np.arange(0, len(pupilSizeL_Trial)/90, 1/90)
        
        #print(len(timeOfGaze_trialNonFilteredPlot), len(pupilSizeL_Trial))
        
        plotPupilSize_checkFilter(pupilSize_Filter['Left'], timeInS_trialFilteredPlot, pupilSizeL_Trial, timeOfGaze_trialNonFilteredPlot, trialNr)
        
    #return timeOfGaze_TrialList, timeInS_List

In [ ]:
def PlotPupilSizeWithEvents_(PupilData_df, EventsTrials, PlotTitle, SubjectAndSessionName):
    
    subjectID = SubjectAndSessionName.split('__')[0]
    
    EventTrials_index = dict()
    EventTrials_index['start'] = [i for i, x in enumerate(EventsTrials['start']) if x] 
    EventTrials_index['end'] = [i for i, x in enumerate(EventsTrials['end']) if x] 
    
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    
    ax.plot(PupilData_df['timeStamp'], PupilData_df['pupilLeft'], 'b', label = 'Left')
    ax.plot(PupilData_df['timeStamp'], PupilData_df['pupilRight'], 'r', label = 'Right')
    
    for start in EventTrials_index['start']:
        ax.plot([PupilData_df['timeStamp'][start], PupilData_df['timeStamp'][start]], [np.min(PupilData_df['pupilLeft']), np.max(PupilData_df['pupilLeft'])], color = 'orange')
    
    for end in EventTrials_index['end']:
        ax.plot([PupilData_df['timeStamp'][end], PupilData_df['timeStamp'][end]], [np.min(PupilData_df['pupilLeft']), np.max(PupilData_df['pupilLeft'])], color = 'green')
    
    ax.set_ylabel('Absolute pupil size [in mm]')
    ax.set_xlabel('Time [in s]')
    plt.legend()
    plt.title('plot {}  for subject  {}'.format(PlotTitle, SubjectAndSessionName))
    
    #pickle.dump(fig, open(r'C:\DTU\Results\201901_Expt\Expt_wTrialsMarked' + '\\' + subjectID + '\\' + 'pupilSize_CompleteExperiment_w' + PlotTitle + SubjectAndSessionName, 'wb'))

    
